# Object Detection with PyTorch and Mask R-CNN 

In this tutorial, you will finetune a pre-trained [Mask R-CNN](https://arxiv.org/abs/1703.06870) model on images from the [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/). The dataset has 170 images with 345 instances of pedestrians.

## Prerequisities

- If you are using an Azure Machine Learning Notebook VM, your environment already meets these prerequisites. Otherwise, go through the [Configuration](https://docs.microsoft.com/azure/machine-learning/how-to-configure-environment) steps to install the Azure Machine Learning Python SDK and [create an Azure ML Workspace](https://docs.microsoft.com/azure/machine-learning/how-to-manage-workspace#create-a-workspace).


In [14]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)


SDK version: 1.0.65
Solving environment: done

## Package Plan ##

  environment location: /home/gopalv/miniconda3

  added / updated specs:
    - cudatoolkit=10.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         153 KB
    cudatoolkit-10.1.243       |       h6bb024c_0       347.4 MB
    ------------------------------------------------------------
                                           Total:       347.6 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-10.1.243-h6bb024c_0

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1d-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_4

The following packages will be SUPERSEDED by a higher-priority 

## Diagnostics

Opt-in diagnostics for better experience, quality, and security in future releases.

In [53]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize a workspace

Initialize a [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`, using the [from_config()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.workspace(class)?view=azure-ml-py#from-config-path-none--auth-none---logger-none---file-name-none-) method.

In [1]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


## Create or attach existing Azure ML Managed Compute

You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/concept-compute-target) for training your model. In this tutorial, we use [Azure ML managed compute](https://docs.microsoft.com/azure/machine-learning/how-to-set-up-training-targets#amlcompute) for our remote training compute resource. Specifically, the below code creates a `STANDARD_NC6` GPU cluster that autoscales from 0 to 4 nodes.

**Creation of Compute takes approximately 5 minutes.** If the Aauzre ML Compute with that name is already in your workspace, this code will skip the creation process. 

As with other Azure servies, there are limits on certain resources associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/how-to-manage-quotas) on the default limits and how to request more quota.

> Note that the below code creates GPU compute. If you instead want to create CPU compute, provide a different VM size to the `vm_size` parameter, such as `STANDARD_D2_V2`.

In [55]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# choose a name for your cluster
cluster_name = 'gpu-cluster'

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-03-06T19:31:29.067000+00:00', 'errors': None, 'creationTime': '2020-03-05T14:55:28.441267+00:00', 'modifiedTime': '2020-03-05T15:05:30.544256+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## Train model on the remote compute

### Create a project directory
Create a directory that will contain all the code from your local machine that you will need access to on the remote resource. This includes the training script an any additional files your training script depends on.

In [2]:
import os

project_folder = './pytorch-peds'

try:
    os.makedirs(project_folder, exist_ok=False)
except FileExistsError:
    print(f'project folder {project_folder} exists, moving on...')

project folder ./pytorch-peds exists, moving on...


Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

### Copy training script and dependencies into project directory

In [57]:
import shutil

shutil.copy('data.py', project_folder)
shutil.copy('model.py', project_folder)
shutil.copy('script.py', project_folder)

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./'+ file + '.py', project_folder)

In [58]:
# !git clone https://github.com/pytorch/vision.git

# !git checkout v0.3.0

# %cd vision
# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../
# %cd ..



### Download data and upload to Azure blob storage

First we download the sample dataset, and extract the images into local storage.

In [12]:
import os
import urllib.request

from zipfile import ZipFile

data_file = './test.zip'
ds_path = 'PennFudanData'

os.makedirs(name=ds_path, exist_ok=True)

urllib.request.urlretrieve('https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip', data_file)
zip = ZipFile(file=data_file)
zip.extractall(path=ds_path)

Annotation  PedMasks	  added-object-list.txt
PNGImages   PennFudanPed  readme.txt


Then, we upload the data files to the datastore associated with this workspace, so that we can access them during training.

In [ ]:
# get the default datastore


ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

ds.upload('./PennFudanPed', target_path=ds_path, overwrite=False)

### Register a dataset


In [6]:
from azureml.core import Dataset

dataset_name = 'penn_ds'
datastore_paths = [(ds, ds_path)]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name=dataset_name,
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'PennFudanPed')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "name": "penn_ds",
    "version": 3,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

### Create an experiment

In [62]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

### Specify dependencies with a custom Dockerfile

There are a number of ways to [use environments](https://docs.microsoft.com/azure/machine-learning/how-to-use-environments) for specifying dependencies during model training. In this case, we use a custom Dockerfile.

In [63]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("dockerfiles/Dockerfile1", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 
my_env.docker.base_image = None
my_env.docker.gpu_support = True
my_env.python.interpreter_path = '/opt/miniconda/bin/python'
my_env.python.user_managed_dependencies = True



WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.
WARNING - 'gpu_support' is no longer necessary; AzureML now automatically detects and uses nvidia docker extension when it is available. It will be removed in a future release.


### Create a ScriptRunConfig

Use the [ScriptRunConfig](https://docs.microsoft.com/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py) class to define your run. Specify the source driectory, compute target, and environment.

In [7]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

model_name = 'pytorch-peds'
output_dir = './outputs/test'
n_epochs = 10

script_args = [
    '--dataset_name', dataset_name,
    '--model_name', model_name,
    '--output_dir', output_dir,
    '--n_epochs', n_epochs,
    '--ds_path', ds_path
]
# Add training script to run config
runconfig = ScriptRunConfig(
    source_directory=project_folder,
    script="script.py",
    arguments=script_args)

# Attach compute target to run config
runconfig.run_config.target = cluster_name

# Uncomment the line below if you want to try this locally first
#runconfig.run_config.target = "local"

# Attach environment to run config
runconfig.run_config.environment = my_env

NameError: name 'cluster_name' is not defined

### Submit your run

In [65]:
# Submit run 
run = experiment.submit(runconfig)

# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1583523377_18df4ee8', 'target': 'gpu-cluster', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '86550606-a30c-485c-bd50-ebfd28c29cfe', 'azureml.git.repository_uri': 'git@github.com:gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'git@github.com:gvashishtha/pytorch-object.git', 'azureml.git.branch': 'register-model', 'mlflow.source.git.branch': 'register-model', 'azureml.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc', 'mlflow.source.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc', 'azureml.git.dirty': 'False', 'AzureML.DerivedImageName': 'azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': ['--dataset_name', 'penn_ds', '--model_name', 'pytorch-peds', '--output_dir', './outputs/test', '--n_epochs', '10'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'Non

### Monitor your run

In [66]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1583523377_18df4ee8
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1583523377_18df4ee8

Streaming azureml-logs/55_azureml-execution-tvmps_6afc19d97c1db353b72a8beae7fbe32dca2625eda4a1e38d7efea51e285c6f06_d.txt

2020-03-06T19:39:14Z Starting output-watcher...
2020-03-06T19:39:14Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d
7ddbc47eeb70: Pulling fs layer
c1bbdc448b72: Pulling fs layer
8c3b70e39044: Pulling fs layer
45d437916d57: Pulling fs layer
d8f1569ddae6: Pulling fs layer
85386706b020: Pulling fs layer
ee9b457b77d0: Pulling fs layer
be4f3343ecd3: Pulling fs layer
30b4effda4fd: Pulling fs layer
b398e882f414: Pulling fs layer
f2e1f2321196: Pulling fs layer
1e87

imgs is ['FudanPed00001.png', 'FudanPed00002.png', 'FudanPed00003.png', 'FudanPed00004.png', 'FudanPed00005.png', 'FudanPed00006.png', 'FudanPed00007.png', 'FudanPed00008.png', 'FudanPed00009.png', 'FudanPed00010.png', 'FudanPed00011.png', 'FudanPed00012.png', 'FudanPed00013.png', 'FudanPed00014.png', 'FudanPed00015.png', 'FudanPed00016.png', 'FudanPed00017.png', 'FudanPed00018.png', 'FudanPed00019.png', 'FudanPed00020.png', 'FudanPed00021.png', 'FudanPed00022.png', 'FudanPed00023.png', 'FudanPed00024.png', 'FudanPed00025.png', 'FudanPed00026.png', 'FudanPed00027.png', 'FudanPed00028.png', 'FudanPed00029.png', 'FudanPed00030.png', 'FudanPed00031.png', 'FudanPed00032.png', 'FudanPed00033.png', 'FudanPed00034.png', 'FudanPed00035.png', 'FudanPed00036.png', 'FudanPed00037.png', 'FudanPed00038.png', 'FudanPed00039.png', 'FudanPed00040.png', 'FudanPed00041.png', 'FudanPed00042.png', 'FudanPed00043.png', 'FudanPed00044.png', 'FudanPed00045.png', 'FudanPed00046.png', 'FudanPed00047.png', 'Fud

 93%|█████████▎| 159M/170M [00:03<00:00, 27.0MB/s]
 95%|█████████▌| 162M/170M [00:04<00:00, 23.3MB/s]
 97%|█████████▋| 166M/170M [00:04<00:00, 26.6MB/s]
 99%|█████████▉| 169M/170M [00:04<00:00, 27.7MB/s]
100%|██████████| 170M/170M [00:04<00:00, 40.6MB/s]
Epoch: [0]  [ 0/60]  eta: 0:02:45  lr: 0.000090  loss: 3.5827 (3.5827)  loss_classifier: 0.7385 (0.7385)  loss_box_reg: 0.1523 (0.1523)  loss_mask: 2.6620 (2.6620)  loss_objectness: 0.0224 (0.0224)  loss_rpn_box_reg: 0.0076 (0.0076)  time: 2.7529  data: 0.1233  max mem: 2303
Epoch: [0]  [10/60]  eta: 0:01:22  lr: 0.000936  loss: 1.5612 (2.1214)  loss_classifier: 0.4486 (0.4978)  loss_box_reg: 0.1826 (0.1906)  loss_mask: 0.9259 (1.4017)  loss_objectness: 0.0224 (0.0208)  loss_rpn_box_reg: 0.0090 (0.0105)  time: 1.6595  data: 0.0214  max mem: 2862
Epoch: [0]  [20/60]  eta: 0:01:03  lr: 0.001783  loss: 0.8715 (1.4321)  loss_classifier: 0.2402 (0.3414)  loss_box_reg: 0.1579 (0.1732)  loss_mask: 0.4010 (0.8839)  loss_objectness: 0.0191 (0.0

Epoch: [2]  [10/60]  eta: 0:01:19  lr: 0.005000  loss: 0.1911 (0.1745)  loss_classifier: 0.0295 (0.0296)  loss_box_reg: 0.0124 (0.0163)  loss_mask: 0.1114 (0.1196)  loss_objectness: 0.0006 (0.0011)  loss_rpn_box_reg: 0.0066 (0.0080)  time: 1.5896  data: 0.0164  max mem: 3597
Epoch: [2]  [20/60]  eta: 0:01:01  lr: 0.005000  loss: 0.1458 (0.1605)  loss_classifier: 0.0253 (0.0249)  loss_box_reg: 0.0089 (0.0125)  loss_mask: 0.1080 (0.1152)  loss_objectness: 0.0004 (0.0008)  loss_rpn_box_reg: 0.0043 (0.0070)  time: 1.5339  data: 0.0069  max mem: 3597
Epoch: [2]  [30/60]  eta: 0:00:47  lr: 0.005000  loss: 0.1585 (0.1812)  loss_classifier: 0.0243 (0.0307)  loss_box_reg: 0.0089 (0.0167)  loss_mask: 0.1133 (0.1241)  loss_objectness: 0.0003 (0.0012)  loss_rpn_box_reg: 0.0076 (0.0086)  time: 1.5709  data: 0.0069  max mem: 3597
Epoch: [2]  [40/60]  eta: 0:00:31  lr: 0.005000  loss: 0.1853 (0.1827)  loss_classifier: 0.0307 (0.0301)  loss_box_reg: 0.0139 (0.0164)  loss_mask: 0.1296 (0.1264)  loss_ob

Epoch: [4]  [10/60]  eta: 0:01:19  lr: 0.000500  loss: 0.1594 (0.1610)  loss_classifier: 0.0216 (0.0211)  loss_box_reg: 0.0115 (0.0132)  loss_mask: 0.1101 (0.1185)  loss_objectness: 0.0004 (0.0007)  loss_rpn_box_reg: 0.0075 (0.0075)  time: 1.5968  data: 0.0166  max mem: 3597
Epoch: [4]  [20/60]  eta: 0:01:01  lr: 0.000500  loss: 0.1594 (0.1625)  loss_classifier: 0.0216 (0.0239)  loss_box_reg: 0.0088 (0.0118)  loss_mask: 0.1101 (0.1191)  loss_objectness: 0.0004 (0.0012)  loss_rpn_box_reg: 0.0057 (0.0066)  time: 1.5403  data: 0.0070  max mem: 3597
Epoch: [4]  [30/60]  eta: 0:00:46  lr: 0.000500  loss: 0.1437 (0.1675)  loss_classifier: 0.0254 (0.0273)  loss_box_reg: 0.0086 (0.0125)  loss_mask: 0.0997 (0.1189)  loss_objectness: 0.0005 (0.0014)  loss_rpn_box_reg: 0.0057 (0.0074)  time: 1.5184  data: 0.0073  max mem: 3597
Epoch: [4]  [40/60]  eta: 0:00:31  lr: 0.000500  loss: 0.1502 (0.1655)  loss_classifier: 0.0256 (0.0275)  loss_box_reg: 0.0092 (0.0121)  loss_mask: 0.1094 (0.1174)  loss_ob

Epoch: [6]  [10/60]  eta: 0:01:25  lr: 0.000050  loss: 0.1393 (0.1461)  loss_classifier: 0.0195 (0.0229)  loss_box_reg: 0.0053 (0.0078)  loss_mask: 0.1032 (0.1095)  loss_objectness: 0.0003 (0.0005)  loss_rpn_box_reg: 0.0057 (0.0053)  time: 1.7019  data: 0.0162  max mem: 3597
Epoch: [6]  [20/60]  eta: 0:01:07  lr: 0.000050  loss: 0.1393 (0.1532)  loss_classifier: 0.0209 (0.0251)  loss_box_reg: 0.0063 (0.0107)  loss_mask: 0.1022 (0.1103)  loss_objectness: 0.0002 (0.0005)  loss_rpn_box_reg: 0.0059 (0.0066)  time: 1.6770  data: 0.0067  max mem: 3597
Epoch: [6]  [30/60]  eta: 0:00:50  lr: 0.000050  loss: 0.1466 (0.1546)  loss_classifier: 0.0227 (0.0268)  loss_box_reg: 0.0094 (0.0103)  loss_mask: 0.1022 (0.1102)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0073 (0.0068)  time: 1.6491  data: 0.0069  max mem: 3597
Epoch: [6]  [40/60]  eta: 0:00:33  lr: 0.000050  loss: 0.1487 (0.1601)  loss_classifier: 0.0253 (0.0269)  loss_box_reg: 0.0094 (0.0112)  loss_mask: 0.1048 (0.1141)  loss_ob

Epoch: [8]  [10/60]  eta: 0:01:20  lr: 0.000050  loss: 0.1616 (0.1546)  loss_classifier: 0.0228 (0.0250)  loss_box_reg: 0.0086 (0.0112)  loss_mask: 0.1125 (0.1117)  loss_objectness: 0.0003 (0.0006)  loss_rpn_box_reg: 0.0048 (0.0061)  time: 1.6187  data: 0.0256  max mem: 3597
Epoch: [8]  [20/60]  eta: 0:01:04  lr: 0.000050  loss: 0.1489 (0.1587)  loss_classifier: 0.0209 (0.0230)  loss_box_reg: 0.0095 (0.0116)  loss_mask: 0.1081 (0.1160)  loss_objectness: 0.0005 (0.0010)  loss_rpn_box_reg: 0.0069 (0.0071)  time: 1.6069  data: 0.0055  max mem: 3597
Epoch: [8]  [30/60]  eta: 0:00:47  lr: 0.000050  loss: 0.1402 (0.1558)  loss_classifier: 0.0206 (0.0235)  loss_box_reg: 0.0095 (0.0118)  loss_mask: 0.1003 (0.1125)  loss_objectness: 0.0004 (0.0009)  loss_rpn_box_reg: 0.0073 (0.0071)  time: 1.5895  data: 0.0068  max mem: 3597
Epoch: [8]  [40/60]  eta: 0:00:32  lr: 0.000050  loss: 0.1486 (0.1641)  loss_classifier: 0.0236 (0.0254)  loss_box_reg: 0.0095 (0.0126)  loss_mask: 0.1091 (0.1175)  loss_ob

{'runId': 'pytorch-peds_1583523377_18df4ee8',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-03-06T19:39:09.795484Z',
 'endTimeUtc': '2020-03-06T20:03:01.126438Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '86550606-a30c-485c-bd50-ebfd28c29cfe',
  'azureml.git.repository_uri': 'git@github.com:gvashishtha/pytorch-object.git',
  'mlflow.source.git.repoURL': 'git@github.com:gvashishtha/pytorch-object.git',
  'azureml.git.branch': 'register-model',
  'mlflow.source.git.branch': 'register-model',
  'azureml.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc',
  'mlflow.source.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc',
  'azureml.git.dirty': 'False',
  'AzureML.DerivedImageName': 'azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '1cd2dd50-ae6d-44f9-81e2-f0

## Test your model

Now that we are done training, let's see how well this model actually performs.

### Get your latest run
First, pull the latest run using `experiment.get_runs()`, which lists runs from `experiment` in reverse chronological order.

In [69]:
from azureml.core import Run

last_successful = next(experiment.get_runs())

In [70]:
last_successful.get_properties()

{'_azureml.ComputeTargetType': 'amlcompute',
 'ContentSnapshotId': '86550606-a30c-485c-bd50-ebfd28c29cfe',
 'azureml.git.repository_uri': 'git@github.com:gvashishtha/pytorch-object.git',
 'mlflow.source.git.repoURL': 'git@github.com:gvashishtha/pytorch-object.git',
 'azureml.git.branch': 'register-model',
 'mlflow.source.git.branch': 'register-model',
 'azureml.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc',
 'mlflow.source.git.commit': '5dffebfe491a17ce3c137ab1c5f308b4f72885cc',
 'azureml.git.dirty': 'False',
 'AzureML.DerivedImageName': 'azureml/azureml_9125fd9b495cfdec8f7bf56c6d28d91d',
 'ProcessInfoFile': 'azureml-logs/process_info.json',
 'ProcessStatusFile': 'azureml-logs/process_status.json'}

### Register your model
Next, register the model from your run.

In [72]:
import os

last_successful.get_file_names()
last_successful.register_model(model_name=model_name, model_path=os.path.join(output_dir, model_name))

Model(workspace=Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo'), name=pytorch-peds, id=pytorch-peds:5, version=5, tags={}, properties={})

### Download your model
Next, download this registered model. Notice how we can initialize the `Model` object with the name of the registered model, rather than a path to the file itself.

In [9]:
from azureml.core import Model

model = Model(workspace=ws, name=model_name)
path = model.download(target_dir=project_folder, exist_ok=True)

### Use your model to make a prediction

Run inferencing on a single test image and display the results.

In [ ]:
!pip install torchvision==0.5.0
!conda install cudatoolkit=10.1 -c pytorch -y

In [10]:
import torch
from azureml.core import Dataset
from data import PennFudanDataset
from script import get_transform

from model import get_instance_segmentation_model
from script import NUM_CLASSES

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

predict_model = get_instance_segmentation_model(NUM_CLASSES)

predict_model.to(device)

predict_model.load_state_dict(torch.load(path, map_location=device))
predict_model.eval()

penn_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)
dataset_test = PennFudanDataset(dataset=penn_ds, transforms=get_transform(train=False), ds_path=ds_path)


# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
predict_model.eval()
with torch.no_grad():
    prediction = predict_model([img.to(device)])

# model = torch.load(path)
#torch.load(model.get_model_path(model_name='outputs/model.pt'))

FileNotFoundError: [Errno 2] No such file or directory: './PennFudanPed/PNGImages'

### Display the input image

In [ ]:
Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())

### Display the predicted mask

In [ ]:
Image.fromarray(prediction[0]['masks'][0, 0].mul(255).byte().cpu().numpy())

In [22]:
import os
import sys
py_dll_path = os.path.join(sys.exec_prefix, 'Library', 'bin')
os.add_dll_directory(py_dll_path)

AttributeError: module 'os' has no attribute 'add_dll_directory'

In [23]:
penn_ds.to_path()

array(['/PennFudanPed/Annotation/FudanPed00001.txt',
       '/PennFudanPed/Annotation/FudanPed00002.txt',
       '/PennFudanPed/Annotation/FudanPed00003.txt',
       '/PennFudanPed/Annotation/FudanPed00004.txt',
       '/PennFudanPed/Annotation/FudanPed00005.txt',
       '/PennFudanPed/Annotation/FudanPed00006.txt',
       '/PennFudanPed/Annotation/FudanPed00007.txt',
       '/PennFudanPed/Annotation/FudanPed00008.txt',
       '/PennFudanPed/Annotation/FudanPed00009.txt',
       '/PennFudanPed/Annotation/FudanPed00010.txt',
       '/PennFudanPed/Annotation/FudanPed00011.txt',
       '/PennFudanPed/Annotation/FudanPed00012.txt',
       '/PennFudanPed/Annotation/FudanPed00013.txt',
       '/PennFudanPed/Annotation/FudanPed00014.txt',
       '/PennFudanPed/Annotation/FudanPed00015.txt',
       '/PennFudanPed/Annotation/FudanPed00016.txt',
       '/PennFudanPed/Annotation/FudanPed00017.txt',
       '/PennFudanPed/Annotation/FudanPed00018.txt',
       '/PennFudanPed/Annotation/FudanPed00019